# Data Loading, Storage, and File Formats

- pandas includes functions for reading muiltiple types of data into DataFrames: read_csv, read_excel, read_sql, read_json, etc
- these read methods have some common arguments: indexing, type inference and data conversion, datetime parsing, iterating, unclean data issues (there are many more for dealing with unclean data)

In [1]:
!cat examples/ex1.csv # cat is the unix shell command for printing file contents to the screen

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('examples/ex1.csv') # read csv will take care of this file format
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
pd.read_table('examples/ex1.csv', delimiter=',') # can also use read_table with a comma delimiter

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
!cat examples/ex2.csv # this data has no headers, how should we handle it?

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
pd.read_csv('examples/ex2.csv', header=None) # ignore them or make them with the 'names' keyword arg

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv('examples/ex1.csv', index_col='message')
# here we specify that the message column should be the index of this dataframe

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [7]:
!cat examples/csv_mindex.csv # here is some data in need of hierarchical idnexing

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [8]:
pd.read_csv('examples/csv_mindex.csv', index_col=['key1','key2']) # we specify the multiple keys like so

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [9]:
!cat examples/ex3.txt # here is a file where the delimiters are whitespace

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [10]:
pd.read_table('examples/ex3.txt', sep='\s+')# here we want to use a regex as a delimiter
# the first column is inferred to be the index of the table

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [11]:
!cat examples/ex4.csv # here is an example where we want to skip some rows

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [12]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3]) # do so with the skiprows arg

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- empty items, NA, and NULL in a file will be read in as NaN (or specify with na_values option)
- the na_values can be a dict with different na values for each column
- see page 172 for many more options for read_csv and read_table

### Reading Text in Pieces

In [13]:
pd.options.display.max_rows = 10 #limit the num of rows to print

In [14]:
pd.read_csv('examples/ex6.csv', nrows=15) # ex file with 10,000 lines; we read only 15

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
10,0.240564,-0.328249,1.252155,1.072796,8
11,0.764018,1.165476,-0.639544,1.495258,R
12,0.571035,-0.310537,0.582437,-0.298765,1
13,2.317658,0.430710,-1.334216,0.199679,P


In [15]:
chunks = pd.read_csv('examples/ex6.csv', chunksize=1000) # specify chunksize to break the data up
chunks # an iterable object of chunks is returned

In [16]:
num = pd.Series([])
for chunk in chunks: # now we can iterate over the chunks, operating on each one as we go
    num=num.add(chunk['key'].value_counts(), fill_value=0)
num.sort_values(ascending=False) # here we just count the num of each key

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

### Writing Data to Text Format

In [17]:
import sys
data = pd.read_csv('examples/ex2.csv')
data.to_csv(sys.stdout, sep='|') # here we write the data to stdout with bar delimiters

|1|2|3|4|hello
0|5|6|7|8|world
1|9|10|11|12|foo


- use the na_rep option to write NaNs as a value besides an empty string
- several other methods of choosing what to write exist

In [18]:
data.to_csv(sys.stdout,index=False ,header=False) # disable headers and index like so

5,6,7,8,world
9,10,11,12,foo


### Working with Delimited Formats

In [19]:
!cat examples/ex7.csv # this example may be tricky to read with the previous methods

"a","b","c"
"1","2","3"
"1","2","3"


In [20]:
import csv
reader = csv.reader(open('examples/ex7.csv'))
lines = list(reader)
head, values = lines[0],lines[1:]
data_dict = {h: v for h,v in zip(head, zip(*values))}
pd.DataFrame(data_dict) # here we converted the csv lines into a dict and then into a DataFrame

,a,b,c
0,1,2,3
1,1,2,3


- we can define our own csv dialect for parsing by creating a subclass of csv.Dialect

### JSON Data
- commonly used to send data via HTTP request
- use Python's json.loads to convert JSON to Python
- use json.dumps to convert Python back to JSON
- or use pandas.read_json to convert formatted JSON into DataFrames or Series
- and convert DataFrames and Series to JSON with their .to_json method

### XML and HTML: Web Scraping

- Python has some libraries for this: lxml, beautifulsoup4, and html5lib
- pandas has similar functionality in its read_html function which tries to read tables and returns a list of dataframes

In [21]:
tables = pd.read_html('examples/fdic_failed_bank_list.html') # searches for <table> tags
failures = tables[0] 
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [22]:
close_timestamps = pd.to_datetime(failures['Closing Date']) # here we just collect data from this column
close_timestamps.dt.year.value_counts() # and get the frequency of each year

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

- we can use lxml.objectify to handle most xml
- use objectify.parse to load the xml and the getroot method to get the document root
- the result is a generator which we can use to access any number of data attributes

### Binary Data Formats

- Python's built-in pickle serialization is an easy and efficient way to store data in binary format
- all pandas objects have a to_pickle method that writes data to disk in pickle format

In [23]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [24]:
frame.to_pickle('examples/frame_pickle') # storing dataframe as a tasty pickle

In [25]:
pd.read_pickle('examples/frame_pickle') # *analyzes pickle*

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- other storage formats: Feather, bcolz, HDF5, MessagePack

### USING HDF5

- HDF5 (hierarchical data format 5) is a good and popular method of storing large numerical arrays
- is awesome: fast, efficient, and parallelizable
- Pandas has HDF5Store which is a class for making a dict-like interface to the compressed data

In [26]:
stored = pd.HDFStore('mydata.h5')
stored # stored now it is just a class

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [27]:
stored['obj1'] = pd.DataFrame({'a':np.random.randn(100)}) # we can store objects like so
stored['obj1'] # it might only like numerical data, as the frame object above gave a TypeError

,a
0,0.311412
1,-0.619286
2,-0.803603
3,-0.279233
4,-1.025635
...,...
95,1.131581
96,0.037402
97,1.135941
98,0.459446


- there are two main storage schemas: fixed and table
- the latter is slower but gives us flexible querying methods: store.put and store.select
- we can pass these queries keyword options for more expressive querying
- we can also use an object's to_hdf method or Panda's read_hdf as a shortcut
- on distributed systems, different storage algorithms may be more appropriate

### Reading Microsoft Excel Files

- deal with Microsoft's stuff using the ExcelFile class or the pd.read_excel function

In [28]:
xlsx = pd.ExcelFile('examples/ex1.xlsx') # here we have an excel file
pd.read_excel(xlsx, 'Sheet1') # look at that! We have excel in a dataframe!

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [29]:
writer = pd.ExcelWriter('examples/ex2.xlsx') # to write to excel, create an ExcelWriter object
frame.to_excel(writer, 'Sheet1') # then call to_excel on it
writer.save() # finally, save it

### Interacting with Web APIs
- check out Python's request package for dealing with APIs

In [30]:
 import requests # Python package for dealing with API communication
url='https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url) # this actually sends the request
resp # this Response obejct contains JSON stings

<Response [200]>

In [31]:
data = resp.json()# and we can use its .json() method to get them as a list of Python dicts
data[0]['title'] # so for the first object, we access its title like so

u'Regression in 0.24: TypeError exception when using dropna on dataframe with categorical index'

- this will be very useful if we want to create interfaces to APIs that return DataFrames

### Interacting with Databases
- pandas has some simple functions for dealing with SQL databases

In [32]:
import sqlite3 # Python's built in SQL package
query = """CREATE TABLE test(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);""" # SQL code
con = sqlite3.connect('mydata.sqlite') # connection? 
try:
    con.execute(query) # execute our query
    con.commit()
    data = [('Atlanta','Georgia',1.25,6),('Tallahassee','Florida',2.6,3)]
    stmt = "INSERT INTO test VALUES(?,?,?,?)"
    con.executemany(stmt,data) # maps stmt to all elements in data
    con.commit()
except:
    print "table test already exists"

table test already exists


- Python has some SQL drivers: PyODBC, psycopg2, MySQLdb, pymssql, etc

In [33]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows # it's common for such selections to return a list of tuples

[(u'Atlanta', u'Georgia', 1.25, 6), (u'Tallahassee', u'Florida', 2.6, 3)]

In [34]:
cursor.description# rows suffices to build a dataframe but we need column names too

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [35]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description]) # and now we can convert to a dataframe

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3


In [36]:
import sqlalchemy as sqla # this package makes working with SQL even easier
db = sqla.create_engine('sqlite:///mydata.sqlite') # just specify the connector like so
pd.read_sql('select * from test',db) # this lets us use Panda's read_sql function

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
